# AdaBoost 

In [2]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [3]:
# load basic libraries
import pandas as pd
import numpy as np
import mne
from pathlib import Path
import pickle
import time

from sklearn.metrics import make_scorer, accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, cross_val_score

%matplotlib widget
import matplotlib
import matplotlib.pyplot as plt

# set directories
# %cd D:/Programy/Anaconda3/Projects/EEG ML project # working directory
%cd D:
pkls = './Pickles/' # objects & variables

D:\Programy\Anaconda3\Projects\EEG ML project


In [4]:
# load split sets, eeg only
with open(pkls +'xy_train.pkl', 'rb') as handle:
    x_train = pickle.load(handle)
    y_train = pickle.load(handle)
y_train.shape
x_train.shape

(6040, 94)

In [4]:
# load x train reduced (post-PCA), eeg only
with open(pkls +'x_train_reduced.pkl', 'rb') as handle:
    x_train_reduced = pickle.load(handle)

In [ ]:
# grid search
cv = 10
n_estimators = 600

param_grid = {'base_estimator__max_depth': [2, 3, 4], # tree's params
              'base_estimator__max_features': ['log2', 'auto'],
              'base_estimator__max_leaf_nodes': [8, 16, 24],
              'base_estimator__min_samples_leaf': [1,2],
              'learning_rate': [0.5, 1]}               # ada's params

scorers = {'accuracy': make_scorer(accuracy_score),
           'f1'      : make_scorer(f1_score, average = 'weighted')}

dt_clf = DecisionTreeClassifier(random_state = 42)
ada_clf = AdaBoostClassifier(base_estimator = dt_clf, n_estimators = n_estimators)

grid = GridSearchCV(ada_clf, param_grid, refit='accuracy', verbose = 1, 
                        scoring = scorers, return_train_score=True, cv = cv, n_jobs = -1)

grid.fit(x_train, y_train.values.ravel())
best_ada = grid.best_estimator_
best_ada

Fitting 10 folds for each of 4 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


In [8]:
best_ada

AdaBoostClassifier(algorithm='SAMME.R',
                   base_estimator=DecisionTreeClassifier(ccp_alpha=0.0,
                                                         class_weight=None,
                                                         criterion='gini',
                                                         max_depth=3,
                                                         max_features=None,
                                                         max_leaf_nodes=None,
                                                         min_impurity_decrease=0.0,
                                                         min_impurity_split=None,
                                                         min_samples_leaf=1,
                                                         min_samples_split=2,
                                                         min_weight_fraction_leaf=0.0,
                                                         presort='deprecated',
                          

In [5]:
# evaluate and save
best_ada_res = round(pd.DataFrame(index = ['gbc'], data  = 
          {'acc mean':[grid.cv_results_['mean_test_accuracy'][grid.best_index_]],
           'acc 2sd': [2*grid.cv_results_['std_test_accuracy'][grid.best_index_]],
           'f1 mean': [grid.cv_results_['mean_test_f1'][grid.best_index_]],
           'f1 2sd' : [2*grid.cv_results_['std_test_f1'][grid.best_index_]]}),4)
best_ada_res

,acc mean,acc 2sd,f1 mean,f1 2sd
gbc,0.5033,0.0391,0.4998,0.04


In [9]:
# train set score for comparison
ada_clf.fit(x_train, y_train.values.ravel())
print("training acc mean: {:.4f}".format(
    ada_clf.score(x_train, y_train)))
print("training f1  mean: {:.4f}".format(
    f1_score(y_train, ada_clf.predict(x_train), average = 'weighted')))

training acc mean: 0.6714
training f1  mean: 0.6703


In [6]:
with open('./Pickles/best_ada.pkl', 'wb') as handle:
    pickle.dump(best_ada, handle)
    pickle.dump(best_ada_res, handle)

NameError: name 'best_ada' is not defined

In [7]:
with open('./Pickles/best_ada.pkl', 'rb') as handle:
    best_ada = pickle.load(handle)
    best_ada_res = pickle.load(handle)